---

_You are currently looking at **version 1.0** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [3]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [59]:
# Import Data
City = pd.read_csv("City_Zhvi_AllHomes.csv")



# Use open() as instead of pd.read_csv() because the latter is too complicated. Tried a long while... 
with open('university_towns.txt') as file:
    ListOfData= []
    for line in file:
        ListOfData.append(line[:-1])
state_town = []


for line in ListOfData:
    if line[-6:]=="[edit]":
        state = line[:-6]
    elif "(" in line:
        town = line[:line.index("(")-1]
        state_town.append([state,town])
    else:
        town = line
        state_town.append([state,town])
    University = pd.DataFrame(state_town,columns=["State","RegionName"])
    
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan","Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State","RegionName"]  )'''

    return University

get_list_of_university_towns()

,State,RegionName
0,Alabama,Auburn
1,Alabama,Florence
2,Alabama,Jacksonville
3,Alabama,Livingston
4,Alabama,Montevallo
5,Alabama,Troy
6,Alabama,Tuscaloosa
7,Alabama,Tuskegee
8,Alaska,Fairbanks
9,Arizona,Flagstaff


In [83]:
# A recession is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.

xlsFile= pd.ExcelFile('gdplev.xls')
# Check the sheet names of the Excel file
# xlsFile.sheet_names
GDP= xlsFile.parse('Sheet1')

# Rename the GDP dataframe and keep only data after 2000q1
GDP.drop(GDP.columns[[0,1,2,3,7]],axis=1, inplace=True)
GDP.columns=["Quarterly","GDP in billions of current dollars","GDP in billions of chained 2009 dollars"]
Pre2000Index = GDP.loc[GDP["Quarterly"]=="2000q1"].index.values.item()
GDP=GDP.ix[Pre2000Index:]

# Reset index (not necessary) and keep only the "Quarterly" and "GDP in billions of current dollars" columns
GDP.reset_index(inplace=True)
GDP=GDP[GDP.columns[[1,2]]]

# Add a new column for quarter over quarter difference. If the value < 0, it marks the beginning of a decline. 
GDP["Change"]=GDP["GDP in billions of current dollars"].diff()
GDP[GDP["Change"]<0]

# Note that there is only one period where there are 2 consecutive declines between quarters, 2008q4 & 2009q1, and 2009q1 &2009q2. 
# 2009q3 and 2009q4 have 2 consecutive quarters of GDP growth.
# Therefore, the recession started on 2008q3 and was recovered in 2009q3.

recessionStart = GDP[GDP["Quarterly"]=="2008q3"]

def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    
    return recessionStart["Quarterly"].item()

get_recession_start()



# # The above solution is hard coded...here's a smarter version:


# quarters = []
# for i in range(len(GDP) - 2):
#     if (GDP.iloc[i]["Change"] < 0) & (GDP.iloc[i+1]["Change"] <0 ):
#         quarters.append(GDP.iloc[i-1][0])
# quarters[0]

# def get_recession_start():
#     '''Returns the year and quarter of the recession start time as a 
#     string value in a format such as 2005q3'''
    
#     return quarters[0]

# get_recession_start()

'2008q3'

In [80]:
quarters = []
for i in range(len(GDP) - 2):
    if (GDP.iloc[i]["Change"] > 0) & (GDP.iloc[i+1]["Change"] >0 ):
        quarters.append(GDP.iloc[i-1][0])

post2008recovery = []

for i in quarters:
    if i > '2008q3':
        post2008recovery.append(i)
post2008recovery

def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
       # Using the 3rd quarter with growing GDP
    return post2008recovery[2]

get_recession_end()

'2009q4'

In [ ]:
# From question 2 & 3 we know that recession took place from 2008q3 to 2009q4
# A recession bottom is the quarter within a recession which had the lowest GDP.

def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    
    return "ANSWER"

In [ ]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    
    return "ANSWER"

In [ ]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    
    return "ANSWER"